# Exercise 7: Multiple Specialized Agents
## Creating Three Finance Agents: Tax Advisor, Investment Analyst, Budget Planner

In this notebook, we'll create three specialized financial agents, each with different tools, temperatures, and expertise areas. We'll learn how to configure agents for specific roles and coordinate their interactions.

## Learning Objectives
- Create specialized agents with different configurations
- Assign specific tools to different agent types
- Configure agent personalities and temperatures
- Build a multi-agent financial advisory system
- Understand agent specialization patterns
- Coordinate multiple agents for complex tasks

Let's build a team of financial experts! 🤖💼📊

## Setup and Imports

First, let's import all the necessary libraries for building our multi-agent system.

In [1]:
# Install required packages
!uv add langchain langchain-openai langchain-community python-dotenv yfinance pandas numpy

Resolved 393 packages in 8ms
Audited 224 packages in 3ms


In [2]:
# Import necessary libraries
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool, StructuredTool, tool
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv
import json
import math
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Optional, Dict, Any, List
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

print("✓ Setup complete!")
print("✓ Ready to build specialized agents!")

✓ Setup complete!
✓ Ready to build specialized agents!


## Part 1: Tax Advisor Tools

Let's start by creating specialized tools for our Tax Advisor agent. These tools will handle tax calculations, deductions, and tax planning scenarios.

In [3]:
# Tax calculation tools for the Tax Advisor agent

class TaxCalculationInput(BaseModel):
    """Input for tax calculation"""
    income: float = Field(description="Annual income in USD")
    filing_status: str = Field(description="Filing status: single, married_joint, married_separate, head_of_household")
    deductions: float = Field(default=0, description="Total deductions amount")

@tool
def calculate_federal_tax(input_data: str) -> str:
    """
    Calculate federal income tax based on 2024 tax brackets.
    
    Args:
        input_data: JSON string with income, filing_status, and deductions
    
    Returns:
        Federal tax calculation details
    """
    try:
        data = json.loads(input_data)
        income = data['income']
        filing_status = data['filing_status']
        deductions = data.get('deductions', 0)
        
        # 2024 standard deductions (simplified)
        standard_deductions = {
            'single': 14600,
            'married_joint': 29200,
            'married_separate': 14600,
            'head_of_household': 21900
        }
        
        # Use higher of standard or itemized deductions
        total_deductions = max(deductions, standard_deductions.get(filing_status, 14600))
        taxable_income = max(0, income - total_deductions)
        
        # 2024 tax brackets for single filers (simplified)
        if filing_status == 'single':
            brackets = [
                (11000, 0.10),
                (44725, 0.12),
                (95375, 0.22),
                (182050, 0.24),
                (231250, 0.32),
                (578125, 0.35),
                (float('inf'), 0.37)
            ]
        else:
            # Simplified - using single brackets for demo
            brackets = [
                (11000, 0.10),
                (44725, 0.12),
                (95375, 0.22),
                (182050, 0.24),
                (231250, 0.32),
                (578125, 0.35),
                (float('inf'), 0.37)
            ]
        
        # Calculate tax
        tax = 0
        prev_bracket = 0
        
        for bracket_limit, rate in brackets:
            if taxable_income <= prev_bracket:
                break
            
            taxable_in_bracket = min(taxable_income, bracket_limit) - prev_bracket
            tax += taxable_in_bracket * rate
            prev_bracket = bracket_limit
            
            if taxable_income <= bracket_limit:
                break
        
        effective_rate = (tax / income * 100) if income > 0 else 0
        
        return f"""
Federal Tax Calculation:
- Gross Income: ${income:,.2f}
- Filing Status: {filing_status}
- Total Deductions: ${total_deductions:,.2f}
- Taxable Income: ${taxable_income:,.2f}
- Federal Tax: ${tax:,.2f}
- Effective Tax Rate: {effective_rate:.2f}%
"""
    
    except Exception as e:
        return f"Error calculating tax: {str(e)}"

# Test the tax calculator
print("Testing Tax Calculator:")
test_input = json.dumps({
    "income": 75000,
    "filing_status": "single",
    "deductions": 5000
})
print(calculate_federal_tax.run(test_input))
print("✓ Tax calculator created!")

Testing Tax Calculator:

Federal Tax Calculation:
- Gross Income: $75,000.00
- Filing Status: single
- Total Deductions: $14,600.00
- Taxable Income: $60,400.00
- Federal Tax: $8,595.50
- Effective Tax Rate: 11.46%

✓ Tax calculator created!


In [4]:
@tool
def optimize_deductions(input_data: str) -> str:
    """
    Analyze and suggest tax deduction optimizations.
    
    Args:
        input_data: JSON string with filing_status and current deductions breakdown
    
    Returns:
        Deduction optimization suggestions
    """
    try:
        data = json.loads(input_data)
        filing_status = data['filing_status']
        current_deductions = data.get('deductions', {})
        
        # 2024 standard deductions
        standard_deductions = {
            'single': 14600,
            'married_joint': 29200,
            'married_separate': 14600,
            'head_of_household': 21900
        }
        
        standard_deduction = standard_deductions.get(filing_status, 14600)
        total_itemized = sum(current_deductions.values())
        
        advice = f"""
Deduction Analysis for {filing_status}:
- Standard Deduction: ${standard_deduction:,.2f}
- Current Itemized Total: ${total_itemized:,.2f}
- Recommendation: {"Itemize" if total_itemized > standard_deduction else "Take Standard"}

Common Deduction Opportunities:
- Charitable donations (up to 60% of AGI)
- State and local taxes (max $10,000)
- Mortgage interest
- Medical expenses (over 7.5% of AGI)
- Business expenses (if applicable)
"""
        
        if total_itemized <= standard_deduction:
            advice += f"\nConsider increasing charitable donations or other deductible expenses by ${(standard_deduction - total_itemized + 1):,.2f} to benefit from itemizing."
        
        return advice
    
    except Exception as e:
        return f"Error optimizing deductions: {str(e)}"

# Test the deduction optimizer
print("Testing Deduction Optimizer:")
test_input = json.dumps({
    "filing_status": "single",
    "deductions": {
        "charitable": 3000,
        "state_local_tax": 8000,
        "mortgage_interest": 2000
    }
})
print(optimize_deductions.run(test_input))
print("✓ Deduction optimizer created!")

Testing Deduction Optimizer:

Deduction Analysis for single:
- Standard Deduction: $14,600.00
- Current Itemized Total: $13,000.00
- Recommendation: Take Standard

Common Deduction Opportunities:
- Charitable donations (up to 60% of AGI)
- State and local taxes (max $10,000)
- Mortgage interest
- Medical expenses (over 7.5% of AGI)
- Business expenses (if applicable)

Consider increasing charitable donations or other deductible expenses by $1,601.00 to benefit from itemizing.
✓ Deduction optimizer created!


## Part 2: Investment Analyst Tools

Now let's create tools for our Investment Analyst agent. These tools will handle portfolio analysis, risk assessment, and investment recommendations.

In [5]:
# Investment analysis tools for the Investment Analyst agent

@tool
def analyze_portfolio(input_data: str) -> str:
    """
    Analyze portfolio allocation and provide recommendations.
    
    Args:
        input_data: JSON string with portfolio holdings and total value
    
    Returns:
        Portfolio analysis and recommendations
    """
    try:
        data = json.loads(input_data)
        portfolio = data['holdings']  # {asset_type: percentage}
        total_value = data.get('total_value', 100000)
        
        # Target allocations based on age/risk tolerance (simplified)
        conservative_target = {'stocks': 40, 'bonds': 50, 'cash': 10}
        moderate_target = {'stocks': 60, 'bonds': 30, 'cash': 10}
        aggressive_target = {'stocks': 80, 'bonds': 15, 'cash': 5}
        
        # Calculate current allocation
        total_percentage = sum(portfolio.values())
        if total_percentage != 100:
            return "Error: Portfolio percentages must sum to 100%"
        
        # Simple risk assessment
        stock_percentage = portfolio.get('stocks', 0)
        risk_level = "Conservative" if stock_percentage < 50 else "Moderate" if stock_percentage < 70 else "Aggressive"
        
        analysis = f"""
Portfolio Analysis:
- Total Value: ${total_value:,.2f}
- Current Risk Level: {risk_level}

Current Allocation:
"""
        
        for asset, percentage in portfolio.items():
            dollar_amount = total_value * percentage / 100
            analysis += f"- {asset.title()}: {percentage}% (${dollar_amount:,.2f})\n"
        
        # Diversification assessment
        analysis += f"\nDiversification Assessment:\n"
        if len(portfolio) < 3:
            analysis += "- Consider diversifying across more asset classes\n"
        if portfolio.get('stocks', 0) > 70:
            analysis += "- High stock allocation - consider risk tolerance\n"
        if portfolio.get('cash', 0) > 20:
            analysis += "- High cash allocation - consider inflation impact\n"
        
        return analysis
    
    except Exception as e:
        return f"Error analyzing portfolio: {str(e)}"

# Test the portfolio analyzer
print("Testing Portfolio Analyzer:")
test_input = json.dumps({
    "holdings": {
        "stocks": 65,
        "bonds": 25,
        "cash": 10
    },
    "total_value": 150000
})
print(analyze_portfolio.run(test_input))
print("✓ Portfolio analyzer created!")

Testing Portfolio Analyzer:

Portfolio Analysis:
- Total Value: $150,000.00
- Current Risk Level: Moderate

Current Allocation:
- Stocks: 65% ($97,500.00)
- Bonds: 25% ($37,500.00)
- Cash: 10% ($15,000.00)

Diversification Assessment:

✓ Portfolio analyzer created!


In [6]:
@tool
def assess_investment_risk(input_data: str) -> str:
    """
    Assess investment risk based on age, timeline, and risk tolerance.
    
    Args:
        input_data: JSON string with age, investment_timeline, and risk_tolerance
    
    Returns:
        Risk assessment and investment strategy recommendations
    """
    try:
        data = json.loads(input_data)
        age = data['age']
        timeline = data['investment_timeline']  # years
        risk_tolerance = data['risk_tolerance']  # low, medium, high
        
        # Rule of thumb: stock allocation = 100 - age (simplified)
        base_stock_allocation = max(20, 100 - age)
        
        # Adjust based on timeline
        if timeline < 5:
            base_stock_allocation *= 0.7  # More conservative for short timeline
        elif timeline > 20:
            base_stock_allocation *= 1.1  # More aggressive for long timeline
        
        # Adjust based on risk tolerance
        risk_multiplier = {'low': 0.8, 'medium': 1.0, 'high': 1.2}
        final_stock_allocation = min(90, base_stock_allocation * risk_multiplier.get(risk_tolerance, 1.0))
        
        bond_allocation = min(70, 100 - final_stock_allocation - 5)  # Leave 5% for cash
        cash_allocation = 100 - final_stock_allocation - bond_allocation
        
        assessment = f"""
Investment Risk Assessment:
- Age: {age} years
- Investment Timeline: {timeline} years
- Risk Tolerance: {risk_tolerance}

Recommended Allocation:
- Stocks: {final_stock_allocation:.0f}%
- Bonds: {bond_allocation:.0f}%
- Cash: {cash_allocation:.0f}%

Strategy Notes:
"""
        
        if timeline < 5:
            assessment += "- Short timeline suggests conservative approach\n"
        if age > 60:
            assessment += "- Consider increasing bond allocation for stability\n"
        if risk_tolerance == 'high' and age < 40:
            assessment += "- Young age + high risk tolerance = growth opportunity\n"
        
        return assessment
    
    except Exception as e:
        return f"Error assessing risk: {str(e)}"

# Test the risk assessor
print("Testing Risk Assessor:")
test_input = json.dumps({
    "age": 35,
    "investment_timeline": 15,
    "risk_tolerance": "medium"
})
print(assess_investment_risk.run(test_input))
print("✓ Risk assessor created!")

Testing Risk Assessor:

Investment Risk Assessment:
- Age: 35 years
- Investment Timeline: 15 years
- Risk Tolerance: medium

Recommended Allocation:
- Stocks: 65%
- Bonds: 30%
- Cash: 5%

Strategy Notes:

✓ Risk assessor created!


## Part 3: Budget Planner Tools

Finally, let's create tools for our Budget Planner agent. These tools will help with budget analysis, expense tracking, and financial goal planning.

In [7]:
# Budget planning tools for the Budget Planner agent

@tool
def analyze_budget(input_data: str) -> str:
    """
    Analyze monthly budget and provide recommendations.
    
    Args:
        input_data: JSON string with monthly income and expenses breakdown
    
    Returns:
        Budget analysis and recommendations
    """
    try:
        data = json.loads(input_data)
        monthly_income = data['monthly_income']
        expenses = data['expenses']  # {category: amount}
        
        total_expenses = sum(expenses.values())
        remaining = monthly_income - total_expenses
        savings_rate = (remaining / monthly_income * 100) if monthly_income > 0 else 0
        
        analysis = f"""
Budget Analysis:
- Monthly Income: ${monthly_income:,.2f}
- Total Expenses: ${total_expenses:,.2f}
- Remaining: ${remaining:,.2f}
- Savings Rate: {savings_rate:.1f}%

Expense Breakdown:
"""
        
        # Sort expenses by amount for better readability
        sorted_expenses = sorted(expenses.items(), key=lambda x: x[1], reverse=True)
        
        for category, amount in sorted_expenses:
            percentage = (amount / monthly_income * 100) if monthly_income > 0 else 0
            analysis += f"- {category.title()}: ${amount:,.2f} ({percentage:.1f}%)\n"
        
        # Provide recommendations based on common budget rules
        analysis += "\nRecommendations:\n"
        
        if savings_rate < 20:
            analysis += "- Consider increasing savings rate to 20%+ for financial security\n"
        
        housing_cost = expenses.get('housing', 0) + expenses.get('rent', 0)
        if housing_cost > monthly_income * 0.3:
            analysis += "- Housing costs exceed 30% of income - consider reducing\n"
        
        if expenses.get('entertainment', 0) > monthly_income * 0.1:
            analysis += "- Entertainment spending is high - consider moderating\n"
        
        if remaining < 0:
            analysis += "- ⚠️ BUDGET DEFICIT - Immediate action needed to reduce expenses\n"
        
        return analysis
    
    except Exception as e:
        return f"Error analyzing budget: {str(e)}"

# Test the budget analyzer
print("Testing Budget Analyzer:")
test_input = json.dumps({
    "monthly_income": 5000,
    "expenses": {
        "housing": 1500,
        "food": 600,
        "transportation": 400,
        "entertainment": 300,
        "utilities": 200,
        "insurance": 150
    }
})
print(analyze_budget.run(test_input))
print("✓ Budget analyzer created!")

Testing Budget Analyzer:

Budget Analysis:
- Monthly Income: $5,000.00
- Total Expenses: $3,150.00
- Remaining: $1,850.00
- Savings Rate: 37.0%

Expense Breakdown:
- Housing: $1,500.00 (30.0%)
- Food: $600.00 (12.0%)
- Transportation: $400.00 (8.0%)
- Entertainment: $300.00 (6.0%)
- Utilities: $200.00 (4.0%)
- Insurance: $150.00 (3.0%)

Recommendations:

✓ Budget analyzer created!


In [8]:
@tool
def plan_financial_goal(input_data: str) -> str:
    """
    Create a savings plan for a specific financial goal.
    
    Args:
        input_data: JSON string with goal_amount, target_months, and current_savings
    
    Returns:
        Financial goal planning analysis
    """
    try:
        data = json.loads(input_data)
        goal_amount = data['goal_amount']
        target_months = data['target_months']
        current_savings = data.get('current_savings', 0)
        
        amount_needed = goal_amount - current_savings
        monthly_saving_needed = amount_needed / target_months if target_months > 0 else 0
        
        # Calculate with interest (simplified 3% annual)
        monthly_rate = 0.03 / 12
        if monthly_rate > 0:
            # Present value of annuity formula
            pv_factor = (1 - (1 + monthly_rate) ** (-target_months)) / monthly_rate
            monthly_with_interest = amount_needed / pv_factor if pv_factor > 0 else monthly_saving_needed
        else:
            monthly_with_interest = monthly_saving_needed
        
        plan = f"""
Financial Goal Planning:
- Goal Amount: ${goal_amount:,.2f}
- Target Timeline: {target_months} months ({target_months/12:.1f} years)
- Current Savings: ${current_savings:,.2f}
- Amount Still Needed: ${amount_needed:,.2f}

Savings Plan:
- Monthly Savings Needed (no interest): ${monthly_saving_needed:,.2f}
- Monthly Savings with 3% interest: ${monthly_with_interest:,.2f}
- Total Monthly Savings: ${monthly_with_interest:,.2f}

Milestones:
"""
        
        # Create quarterly milestones
        for quarter in range(1, min(5, target_months // 3 + 1)):
            months_elapsed = quarter * 3
            if months_elapsed <= target_months:
                saved_by_quarter = current_savings + (monthly_with_interest * months_elapsed)
                plan += f"- Month {months_elapsed}: ${saved_by_quarter:,.2f}\n"
        
        # Tips based on timeline
        plan += "\nTips:\n"
        if target_months < 12:
            plan += "- Short timeline - consider high-yield savings account\n"
        elif target_months < 36:
            plan += "- Medium timeline - consider CDs or conservative investments\n"
        else:
            plan += "- Long timeline - consider growth investments\n"
        
        return plan
    
    except Exception as e:
        return f"Error planning goal: {str(e)}"

# Test the goal planner
print("Testing Goal Planner:")
test_input = json.dumps({
    "goal_amount": 20000,
    "target_months": 24,
    "current_savings": 5000
})
print(plan_financial_goal.run(test_input))
print("✓ Goal planner created!")

Testing Goal Planner:

Financial Goal Planning:
- Goal Amount: $20,000.00
- Target Timeline: 24 months (2.0 years)
- Current Savings: $5,000.00
- Amount Still Needed: $15,000.00

Savings Plan:
- Monthly Savings Needed (no interest): $625.00
- Monthly Savings with 3% interest: $644.72
- Total Monthly Savings: $644.72

Milestones:
- Month 3: $6,934.15
- Month 6: $8,868.31
- Month 9: $10,802.46
- Month 12: $12,736.62

Tips:
- Medium timeline - consider CDs or conservative investments

✓ Goal planner created!


## Part 4: Creating Specialized Agents

Now let's create our three specialized agents, each with different configurations, temperatures, and tool sets. Each agent will have its own personality and expertise area.

In [9]:
# Create the Tax Advisor Agent
def create_tax_advisor_agent():
    """Create a specialized Tax Advisor agent with conservative temperature and tax tools."""
    
    # Conservative LLM for precise tax calculations
    tax_llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.1,  # Very conservative for tax calculations
        api_key=os.getenv("OPENAI_API_KEY")
    )
    
    # Tax advisor specific tools
    tax_tools = [calculate_federal_tax, optimize_deductions]
    
    # Create specialized prompt for tax advisor
    tax_advisor_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a professional Tax Advisor with expertise in US federal tax law. 
        You provide accurate, conservative tax advice and calculations. 
        
        Key principles:
        - Always prioritize accuracy over speed
        - Provide conservative estimates to avoid tax issues
        - Explain tax implications clearly
        - Reference current tax laws and brackets
        - Suggest legitimate tax optimization strategies
        
        Your tone should be professional, precise, and educational."""),
        
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create the tax advisor agent
    tax_agent = create_tool_calling_agent(tax_llm, tax_tools, tax_advisor_prompt)
    
    # Create executor
    tax_executor = AgentExecutor(
        agent=tax_agent,
        tools=tax_tools,
        verbose=True,
        max_iterations=3
    )
    
    return tax_executor

# Test the Tax Advisor
print("Creating Tax Advisor Agent...")
tax_advisor = create_tax_advisor_agent()
print("✓ Tax Advisor Agent created!")

# Test the tax advisor with a simple question
print("\nTesting Tax Advisor:")
response = tax_advisor.invoke({
    "input": "I make $80,000 per year and I'm single. Can you calculate my federal tax and suggest ways to optimize my deductions?"
})
print(f"Tax Advisor Response: {response['output']}")
print("✓ Tax Advisor tested successfully!")

Creating Tax Advisor Agent...
✓ Tax Advisor Agent created!

Testing Tax Advisor:


> Entering new AgentExecutor chain...

Invoking: `calculate_federal_tax` with `{'input_data': '{"income":80000,"filing_status":"single","deductions":0}'}`



Federal Tax Calculation:
- Gross Income: $80,000.00
- Filing Status: single
- Total Deductions: $14,600.00
- Taxable Income: $65,400.00
- Federal Tax: $9,695.50
- Effective Tax Rate: 12.12%



Invoking: `optimize_deductions` with `{'input_data': '{"filing_status":"single","current_deductions":{"standard_deduction":12200,"other_deductions":2400}}'}`



Deduction Analysis for single:
- Standard Deduction: $14,600.00
- Current Itemized Total: $0.00
- Recommendation: Take Standard

Common Deduction Opportunities:
- Charitable donations (up to 60% of AGI)
- State and local taxes (max $10,000)
- Mortgage interest
- Medical expenses (over 7.5% of AGI)
- Business expenses (if applicable)

Consider increasing charitable donations or other deductible expenses by $14,601.00 to benefit from itemizing.Based on your income of $80,000 as a single filer, here are the tax details:
- Gross Income: $80,000.00
- Total Deductions: $14,600.00
- Taxable Income: $65,400.00
- Federal Tax: $9,695.50
- Effective Tax Rate: 12.12%

To optimize your deductions, it's recommended that you take the standard deduction of $14,600. However, if you have additional deductible expenses such as charitable donatio

In [10]:
# Create the Investment Analyst Agent
def create_investment_analyst_agent():
    """Create a specialized Investment Analyst agent with moderate temperature and investment tools."""
    
    # Moderate LLM for balanced investment analysis
    investment_llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.3,  # Moderate temperature for creative but grounded analysis
        api_key=os.getenv("OPENAI_API_KEY")
    )
    
    # Investment analyst specific tools
    investment_tools = [analyze_portfolio, assess_investment_risk]
    
    # Create specialized prompt for investment analyst
    investment_analyst_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a skilled Investment Analyst with expertise in portfolio management and risk assessment.
        You provide data-driven investment advice and portfolio analysis.
        
        Key principles:
        - Balance risk and reward considerations
        - Consider time horizon and risk tolerance
        - Provide diversification recommendations
        - Explain investment concepts clearly
        - Base recommendations on solid financial principles
        - Consider market conditions and economic factors
        
        Your tone should be analytical, insightful, and forward-thinking."""),
        
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create the investment analyst agent
    investment_agent = create_tool_calling_agent(investment_llm, investment_tools, investment_analyst_prompt)
    
    # Create executor
    investment_executor = AgentExecutor(
        agent=investment_agent,
        tools=investment_tools,
        verbose=True,
        max_iterations=3
    )
    
    return investment_executor

# Test the Investment Analyst
print("Creating Investment Analyst Agent...")
investment_analyst = create_investment_analyst_agent()
print("✓ Investment Analyst Agent created!")

# Test the investment analyst with a portfolio question
print("\nTesting Investment Analyst:")
response = investment_analyst.invoke({
    "input": "I'm 30 years old with medium risk tolerance and want to invest for 20 years. I currently have 70% stocks, 20% bonds, and 10% cash. What's your analysis?"
})
print(f"Investment Analyst Response: {response['output']}")
print("✓ Investment Analyst tested successfully!")

Creating Investment Analyst Agent...
✓ Investment Analyst Agent created!

Testing Investment Analyst:


> Entering new AgentExecutor chain...



Invoking: `assess_investment_risk` with `{'input_data': '{"age":30,"investment_timeline":20,"risk_tolerance":"medium"}'}`



Investment Risk Assessment:
- Age: 30 years
- Investment Timeline: 20 years
- Risk Tolerance: medium

Recommended Allocation:
- Stocks: 70%
- Bonds: 25%
- Cash: 5%

Strategy Notes:

Invoking: `analyze_portfolio` with `{'input_data': '{"holdings":{"stocks":70,"bonds":20,"cash":10},"total_value":100}'}`



Portfolio Analysis:
- Total Value: $100.00
- Current Risk Level: Aggressive

Current Allocation:
- Stocks: 70% ($70.00)
- Bonds: 20% ($20.00)
- Cash: 10% ($10.00)

Diversification Assessment:
Based on your age of 30, medium risk tolerance, and 20-year investment timeline, here are the recommendations:

### Investment Risk Assessment:
- **Recommended Allocation:**
  - Stocks: 70%
  - Bonds: 25%
  - Cash: 5%

### Portfolio Analysis:
- **Current Allocation:**
  - Stocks: 70%
  - Bonds: 20%
  - Cash: 10%
- **Current Risk Level:** Aggressive

### Strategy Notes:
- Yo

In [11]:
# Create the Budget Planner Agent
def create_budget_planner_agent():
    """Create a specialized Budget Planner agent with higher temperature and budgeting tools."""
    
    # Higher temperature LLM for creative budgeting solutions
    budget_llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.5,  # Higher temperature for creative budgeting ideas
        api_key=os.getenv("OPENAI_API_KEY")
    )
    
    # Budget planner specific tools
    budget_tools = [analyze_budget, plan_financial_goal]
    
    # Create specialized prompt for budget planner
    budget_planner_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an enthusiastic Budget Planner and Financial Coach who helps people achieve their financial goals.
        You provide practical, motivational budgeting advice and financial planning strategies.
        
        Key principles:
        - Focus on practical, actionable budgeting advice
        - Motivate clients to stick to their financial goals
        - Provide creative solutions for saving money
        - Break down complex goals into manageable steps
        - Emphasize the importance of emergency funds
        - Celebrate small wins and progress
        
        Your tone should be encouraging, practical, and goal-oriented."""),
        
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create the budget planner agent
    budget_agent = create_tool_calling_agent(budget_llm, budget_tools, budget_planner_prompt)
    
    # Create executor
    budget_executor = AgentExecutor(
        agent=budget_agent,
        tools=budget_tools,
        verbose=True,
        max_iterations=3
    )
    
    return budget_executor

# Test the Budget Planner
print("Creating Budget Planner Agent...")
budget_planner = create_budget_planner_agent()
print("✓ Budget Planner Agent created!")

# Test the budget planner with a budgeting question
print("\nTesting Budget Planner:")
response = budget_planner.invoke({
    "input": "I earn $4,000 per month and want to save for a $15,000 car in 18 months. Can you help me analyze my budget and create a savings plan?"
})
print(f"Budget Planner Response: {response['output']}")
print("✓ Budget Planner tested successfully!")

Creating Budget Planner Agent...
✓ Budget Planner Agent created!

Testing Budget Planner:


> Entering new AgentExecutor chain...



Invoking: `analyze_budget` with `{'input_data': '{"monthly_income":4000,"expenses":{"rent":1200,"utilities":200,"groceries":300,"transportation":200,"entertainment":100,"savings":0}}'}`



Budget Analysis:
- Monthly Income: $4,000.00
- Total Expenses: $2,000.00
- Remaining: $2,000.00
- Savings Rate: 50.0%

Expense Breakdown:
- Rent: $1,200.00 (30.0%)
- Groceries: $300.00 (7.5%)
- Utilities: $200.00 (5.0%)
- Transportation: $200.00 (5.0%)
- Entertainment: $100.00 (2.5%)
- Savings: $0.00 (0.0%)

Recommendations:

Invoking: `plan_financial_goal` with `{'input_data': '{"goal_amount":15000,"target_months":18,"current_savings":0}'}`



Financial Goal Planning:
- Goal Amount: $15,000.00
- Target Timeline: 18 months (1.5 years)
- Current Savings: $0.00
- Amount Still Needed: $15,000.00

Savings Plan:
- Monthly Savings Needed (no interest): $833.33
- Monthly Savings with 3% interest: $853.27
- Total Monthly Savings: $853.27

Milestones:
- Month 3: $2,559.80
- Month 6: $5,119.59
- Month 9: $7,6

## Part 5: Multi-Agent Coordination

Now let's create a system that coordinates all three agents to provide comprehensive financial advice. We'll build a financial advisory team that works together.

In [12]:
# Multi-Agent Financial Advisory System
class FinancialAdvisoryTeam:
    """A coordinated team of financial agents working together."""
    
    def __init__(self):
        """Initialize all three specialized agents."""
        self.tax_advisor = create_tax_advisor_agent()
        self.investment_analyst = create_investment_analyst_agent()
        self.budget_planner = create_budget_planner_agent()
        
        # Agent routing based on keywords
        self.agent_routing = {
            'tax': self.tax_advisor,
            'deduction': self.tax_advisor,
            'irs': self.tax_advisor,
            'investment': self.investment_analyst,
            'portfolio': self.investment_analyst,
            'stocks': self.investment_analyst,
            'bonds': self.investment_analyst,
            'risk': self.investment_analyst,
            'budget': self.budget_planner,
            'savings': self.budget_planner,
            'expenses': self.budget_planner,
            'goal': self.budget_planner
        }
    
    def route_question(self, question: str):
        """Route question to the most appropriate agent."""
        question_lower = question.lower()
        
        # Check for keywords to determine the best agent
        for keyword, agent in self.agent_routing.items():
            if keyword in question_lower:
                return agent
        
        # Default to budget planner for general financial questions
        return self.budget_planner
    
    def get_comprehensive_advice(self, client_profile: dict):
        """Get advice from all three agents for a comprehensive financial plan."""
        
        print("🔍 Analyzing your financial situation with our expert team...")
        results = {}
        
        # Tax Analysis
        if 'income' in client_profile and 'filing_status' in client_profile:
            print("\n📊 TAX ADVISOR ANALYSIS:")
            tax_question = f"Please analyze the tax situation for someone with ${client_profile['income']} income, filing as {client_profile['filing_status']}."
            results['tax_analysis'] = self.tax_advisor.invoke({"input": tax_question})['output']
        
        # Investment Analysis
        if 'age' in client_profile and 'risk_tolerance' in client_profile:
            print("\n📈 INVESTMENT ANALYST ANALYSIS:")
            investment_question = f"Please provide investment advice for a {client_profile['age']} year old with {client_profile['risk_tolerance']} risk tolerance."
            results['investment_analysis'] = self.investment_analyst.invoke({"input": investment_question})['output']
        
        # Budget Analysis
        if 'monthly_income' in client_profile:
            print("\n💰 BUDGET PLANNER ANALYSIS:")
            budget_question = f"Please help with budget planning for someone earning ${client_profile['monthly_income']} per month."
            results['budget_analysis'] = self.budget_planner.invoke({"input": budget_question})['output']
        
        return results
    
    def ask_team(self, question: str):
        """Ask a question and route it to the appropriate agent."""
        print(f"🤖 Routing your question to the best expert...")
        
        # Route to appropriate agent
        selected_agent = self.route_question(question)
        
        # Determine which agent was selected
        agent_name = "Budget Planner"  # default
        if selected_agent == self.tax_advisor:
            agent_name = "Tax Advisor"
        elif selected_agent == self.investment_analyst:
            agent_name = "Investment Analyst"
        
        print(f"📋 Selected Expert: {agent_name}")
        
        # Get response from selected agent
        response = selected_agent.invoke({"input": question})
        return {
            "agent": agent_name,
            "response": response['output']
        }

# Create the advisory team
print("Creating Financial Advisory Team...")
advisory_team = FinancialAdvisoryTeam()
print("✓ Financial Advisory Team assembled!")
print("  - Tax Advisor (Temperature: 0.1 - Conservative)")
print("  - Investment Analyst (Temperature: 0.3 - Moderate)")
print("  - Budget Planner (Temperature: 0.5 - Creative)")
print("✓ Multi-agent system ready!")

Creating Financial Advisory Team...
✓ Financial Advisory Team assembled!
  - Tax Advisor (Temperature: 0.1 - Conservative)
  - Investment Analyst (Temperature: 0.3 - Moderate)
  - Budget Planner (Temperature: 0.5 - Creative)
✓ Multi-agent system ready!


## Part 6: Multi-Agent Demonstration

Let's test our multi-agent system with different types of questions and see how they get routed to the appropriate specialists.

In [13]:
# Test 1: Tax-related question (should route to Tax Advisor)
print("=== TEST 1: Tax Question ===")
tax_question = "I have $90,000 in income and want to maximize my tax deductions. What should I do?"
result1 = advisory_team.ask_team(tax_question)
print(f"Agent: {result1['agent']}")
print(f"Response: {result1['response']}")
print()

=== TEST 1: Tax Question ===
🤖 Routing your question to the best expert...
📋 Selected Expert: Tax Advisor


> Entering new AgentExecutor chain...



Invoking: `calculate_federal_tax` with `{'input_data': '{"income":90000,"filing_status":"single","deductions":0}'}`



Federal Tax Calculation:
- Gross Income: $90,000.00
- Filing Status: single
- Total Deductions: $14,600.00
- Taxable Income: $75,400.00
- Federal Tax: $11,895.50
- Effective Tax Rate: 13.22%

Invoking: `optimize_deductions` with `{'input_data': '{"filing_status":"single","current_deductions":{"standard_deduction":12200,"charitable_contributions":0,"mortgage_interest":0,"medical_expenses":0,"other_deductions":0}}'}`



Deduction Analysis for single:
- Standard Deduction: $14,600.00
- Current Itemized Total: $0.00
- Recommendation: Take Standard

Common Deduction Opportunities:
- Charitable donations (up to 60% of AGI)
- State and local taxes (max $10,000)
- Mortgage interest
- Medical expenses (over 7.5% of AGI)
- Business expenses (if applicable)

Consider increasing charitable donations or other deductible expenses by $14,601.00 to benefit from itemizing.Based on you

In [14]:
# Test 2: Investment question (should route to Investment Analyst)
print("=== TEST 2: Investment Question ===")
investment_question = "I'm 25 years old and want to build a diversified portfolio. What's your recommendation?"
result2 = advisory_team.ask_team(investment_question)
print(f"Agent: {result2['agent']}")
print(f"Response: {result2['response']}")
print()

# Test 3: Budget question (should route to Budget Planner)
print("=== TEST 3: Budget Question ===")
budget_question = "I need help creating a monthly budget and saving for an emergency fund."
result3 = advisory_team.ask_team(budget_question)
print(f"Agent: {result3['agent']}")
print(f"Response: {result3['response']}")
print()

=== TEST 2: Investment Question ===
🤖 Routing your question to the best expert...
📋 Selected Expert: Investment Analyst


> Entering new AgentExecutor chain...



Invoking: `assess_investment_risk` with `{'input_data': '{"age":25,"investment_timeline":"long_term","risk_tolerance":"high"}'}`


Error assessing risk: '<' not supported between instances of 'str' and 'int'
Invoking: `assess_investment_risk` with `{'input_data': '{"age":25,"investment_timeline":"long_term","risk_tolerance":"high"}'}`
responded: It seems there was an error in assessing your investment risk. Let me try again with the correct input parameters.

Error assessing risk: '<' not supported between instances of 'str' and 'int'
Invoking: `assess_investment_risk` with `{'input_data': '{"age":25,"investment_timeline":"long_term","risk_tolerance":"high"}'}`
responded: It appears there is still an issue with assessing your investment risk. Let me adjust the input parameters and try again.

Error assessing risk: '<' not supported between instances of 'str' and 'int'

> Finished chain.
Agent: Investment Analyst
Response: Agent stopped due to max iterations.

=== TEST 3: Budget Questi

In [15]:
# Test 4: Comprehensive Analysis (all agents working together)
print("=== TEST 4: Comprehensive Financial Analysis ===")
print("Getting advice from all three agents for a complete financial picture...")

# Sample client profile
client_profile = {
    'income': 70000,
    'filing_status': 'single',
    'age': 28,
    'risk_tolerance': 'medium',
    'monthly_income': 5800
}

# Get comprehensive advice
comprehensive_results = advisory_team.get_comprehensive_advice(client_profile)

print("\n" + "="*50)
print("COMPREHENSIVE FINANCIAL ANALYSIS RESULTS")
print("="*50)

for analysis_type, result in comprehensive_results.items():
    print(f"\n{analysis_type.upper()}:")
    print("-" * 40)
    print(result)
    print()

print("✓ Multi-agent financial advisory system demonstration complete!")

=== TEST 4: Comprehensive Financial Analysis ===
Getting advice from all three agents for a complete financial picture...
🔍 Analyzing your financial situation with our expert team...

📊 TAX ADVISOR ANALYSIS:


> Entering new AgentExecutor chain...



Invoking: `calculate_federal_tax` with `{'input_data': '{"income":70000,"filing_status":"single","deductions":0}'}`



Federal Tax Calculation:
- Gross Income: $70,000.00
- Filing Status: single
- Total Deductions: $14,600.00
- Taxable Income: $55,400.00
- Federal Tax: $7,495.50
- Effective Tax Rate: 10.71%
Based on the analysis of a single filer with a $70,000 income, the federal tax calculation is as follows:
- Gross Income: $70,000.00
- Total Deductions: $14,600.00
- Taxable Income: $55,400.00
- Federal Tax Owed: $7,495.50
- Effective Tax Rate: 10.71%

If you have any specific deductions or expenses that you can claim, it might help reduce your taxable income and lower your tax liability. Would you like me to provide suggestions on optimizing your deductions to potentially reduce your tax burden further?

> Finished chain.

📈 INVESTMENT ANALYST ANALYSIS:


> Entering new AgentExecutor chain...

Invoking: `assess_investment_risk` with `{'input_data': '{"age":28,"investment_timeline"

## Summary: Multi-Agent System Key Concepts

Let's summarize what we've learned about creating specialized agents and coordinating them in a multi-agent system.

In [16]:
# 📋 KEY CONCEPTS SUMMARY

print("🎯 MULTI-AGENT SYSTEM CONCEPTS LEARNED:")
print("="*50)

concepts = {
    "Agent Specialization": {
        "description": "Each agent has specific expertise and tools",
        "example": "Tax Advisor only handles tax calculations and deductions"
    },
    "Temperature Configuration": {
        "description": "Different temperatures for different agent personalities",
        "example": "0.1 (conservative) for Tax, 0.3 (moderate) for Investment, 0.5 (creative) for Budget"
    },
    "Tool Assignment": {
        "description": "Agents only have access to tools relevant to their expertise",
        "example": "Investment Analyst can't access tax calculation tools"
    },
    "Smart Routing": {
        "description": "Questions are automatically routed to the best specialist",
        "example": "Keywords like 'tax', 'portfolio', 'budget' determine routing"
    },
    "Coordination": {
        "description": "Multiple agents can work together on complex problems",
        "example": "Comprehensive financial analysis uses all three agents"
    }
}

for concept, details in concepts.items():
    print(f"\n🔹 {concept}:")
    print(f"   {details['description']}")
    print(f"   Example: {details['example']}")

print("\n" + "="*50)
print("✓ Multi-agent system mastered!")

🎯 MULTI-AGENT SYSTEM CONCEPTS LEARNED:

🔹 Agent Specialization:
   Each agent has specific expertise and tools
   Example: Tax Advisor only handles tax calculations and deductions

🔹 Temperature Configuration:
   Different temperatures for different agent personalities
   Example: 0.1 (conservative) for Tax, 0.3 (moderate) for Investment, 0.5 (creative) for Budget

🔹 Tool Assignment:
   Agents only have access to tools relevant to their expertise
   Example: Investment Analyst can't access tax calculation tools

🔹 Smart Routing:
   Questions are automatically routed to the best specialist
   Example: Keywords like 'tax', 'portfolio', 'budget' determine routing

🔹 Coordination:
   Multiple agents can work together on complex problems
   Example: Comprehensive financial analysis uses all three agents

✓ Multi-agent system mastered!


In [17]:
# 🎯 PRACTICAL SUMMARY EXAMPLE
print("="*60)
print("PRACTICAL EXAMPLE: Building Your Own Multi-Agent System")
print("="*60)

print("""
To create your own multi-agent system, follow these steps:

1. 🛠️ DEFINE SPECIALIZED TOOLS
   - Create focused tools for each domain
   - Keep tools simple and single-purpose
   - Test each tool individually

2. 🤖 CREATE SPECIALIZED AGENTS
   - Configure different temperatures for different personalities
   - Assign only relevant tools to each agent
   - Create domain-specific prompts

3. 🔄 BUILD COORDINATION SYSTEM
   - Implement smart routing based on keywords/context
   - Create a coordinator class to manage all agents
   - Enable agents to work together on complex tasks

4. 🧪 TEST AND ITERATE
   - Test individual agents first
   - Test routing logic
   - Test multi-agent coordination
   - Refine based on results

Example Agent Configurations:
""")

agent_configs = {
    "Tax Advisor": {
        "temperature": 0.1,
        "personality": "Conservative, Precise",
        "tools": ["tax_calculator", "deduction_optimizer"],
        "keywords": ["tax", "deduction", "irs"]
    },
    "Investment Analyst": {
        "temperature": 0.3,
        "personality": "Analytical, Balanced",
        "tools": ["portfolio_analyzer", "risk_assessor"],
        "keywords": ["investment", "portfolio", "stocks"]
    },
    "Budget Planner": {
        "temperature": 0.5,
        "personality": "Creative, Motivational",
        "tools": ["budget_analyzer", "goal_planner"],
        "keywords": ["budget", "savings", "expenses"]
    }
}

for agent_name, config in agent_configs.items():
    print(f"\n🤖 {agent_name}:")
    print(f"   Temperature: {config['temperature']}")
    print(f"   Personality: {config['personality']}")
    print(f"   Tools: {', '.join(config['tools'])}")
    print(f"   Keywords: {', '.join(config['keywords'])}")

print("\n" + "="*60)
print("🎉 You now have all the tools to build specialized multi-agent systems!")
print("="*60)

PRACTICAL EXAMPLE: Building Your Own Multi-Agent System

To create your own multi-agent system, follow these steps:

1. 🛠️ DEFINE SPECIALIZED TOOLS
   - Create focused tools for each domain
   - Keep tools simple and single-purpose
   - Test each tool individually

2. 🤖 CREATE SPECIALIZED AGENTS
   - Configure different temperatures for different personalities
   - Assign only relevant tools to each agent
   - Create domain-specific prompts

3. 🔄 BUILD COORDINATION SYSTEM
   - Implement smart routing based on keywords/context
   - Create a coordinator class to manage all agents
   - Enable agents to work together on complex tasks

4. 🧪 TEST AND ITERATE
   - Test individual agents first
   - Test routing logic
   - Test multi-agent coordination
   - Refine based on results

Example Agent Configurations:


🤖 Tax Advisor:
   Temperature: 0.1
   Personality: Conservative, Precise
   Tools: tax_calculator, deduction_optimizer
   Keywords: tax, deduction, irs

🤖 Investment Analyst:
   Temper

In [18]:
# 🎮 INTERACTIVE EXAMPLE: Try Your Own Questions!
print("="*50)
print("TRY IT YOURSELF!")
print("="*50)

# Example questions you can try
example_questions = [
    "How can I reduce my tax liability if I make $85,000?",
    "Should I invest in stocks or bonds at age 45?",
    "Help me save $10,000 for a vacation in 12 months",
    "What's the best portfolio allocation for retirement?",
    "How do I optimize my deductions for maximum tax savings?"
]

print("Here are some example questions you can ask our advisory team:")
print()

for i, question in enumerate(example_questions, 1):
    print(f"{i}. {question}")

print("\n" + "="*50)
print("🎯 CHALLENGE: Try asking these questions to see the routing in action!")
print("="*50)

# Quick example - uncomment to try
# sample_question = "I want to invest $50,000 but I'm not sure about the risks"
# result = advisory_team.ask_team(sample_question)
# print(f"Routed to: {result['agent']}")
# print(f"Response: {result['response'][:200]}...")  # First 200 chars

print("\n✅ Exercise 7 Complete!")
print("You've successfully built a multi-agent financial advisory system!")
print("🎉 Next: Try building your own specialized agents for different domains!")

TRY IT YOURSELF!
Here are some example questions you can ask our advisory team:

1. How can I reduce my tax liability if I make $85,000?
2. Should I invest in stocks or bonds at age 45?
3. Help me save $10,000 for a vacation in 12 months
4. What's the best portfolio allocation for retirement?
5. How do I optimize my deductions for maximum tax savings?

🎯 CHALLENGE: Try asking these questions to see the routing in action!

✅ Exercise 7 Complete!
You've successfully built a multi-agent financial advisory system!
🎉 Next: Try building your own specialized agents for different domains!
